In [82]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report


In [83]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.datasets import make_blobs
from collections import Counter
from path import Path
import hvplot.pandas


# importing files for w/l

In [84]:
df_2019 = pd.read_csv(Path("random.txt"))
df_2019 = df_2019[["Opp", "Result", "Starting Pitcher", "Opposing Pitcher"]]

true_w = df_2019["Result"].str.contains('W', na=True)
count = 0
Result = []
for row in df_2019.index:
    if true_w[row]:
        Result.append(1)
    else:
        Result.append(0)
        
df_2019["Result"] = Result
df_2019 = df_2019.set_index("Opp")

df_2019.head()

,Result,Starting Pitcher,Opposing Pitcher
Opp,,,
TBA,1,Justin Verlander,Blake Snell
TBA,0,Gerrit Cole,Charlie Morton
TBA,0,Collin McHugh,Tyler Glasnow
TBA,0,Wade Miley,Yonny Chirinos
TEX,1,Brad Peacock,Drew Smyly


In [85]:
df_2018 = pd.read_csv(Path("2018wl.txt"))
df_2018 = df_2018[["Opp", "Result", "Starting Pitcher", "Opposing Pitcher"]]

true_w = df_2018["Result"].str.contains('W', na=True)
count = 0
Result = []
for row in df_2018.index:
    if true_w[row]:
        Result.append(1)
    else:
        Result.append(0)
        
df_2018["Result"] = Result
df_2018 = df_2018.set_index("Opp")

df_2018.head()

,Result,Starting Pitcher,Opposing Pitcher
Opp,,,
TEX,1,Justin Verlander,Cole Hamels
TEX,0,Dallas Keuchel,Doug Fister
TEX,1,Lance McCullers,Matt Moore
TEX,1,Gerrit Cole,Mike Minor
BAL,1,Charlie Morton,Chris Tillman


In [86]:
df_combined = pd.concat([df_2019,df_2018])
df_combined

,Result,Starting Pitcher,Opposing Pitcher
Opp,,,
TBA,1,Justin Verlander,Blake Snell
TBA,0,Gerrit Cole,Charlie Morton
TBA,0,Collin McHugh,Tyler Glasnow
TBA,0,Wade Miley,Yonny Chirinos
TEX,1,Brad Peacock,Drew Smyly
...,...,...,...
TOR,0,Chris Devenski,Sean Reid-Foley
BAL,1,Gerrit Cole,David Hess
BAL,1,Justin Verlander,Dylan Bundy


# importing pitching data

In [87]:
df_pitch_2019 = pd.read_csv(Path("mlb-player-stats-P.csv"))
df_pitch_2019.set_index(["Player"], inplace = True)
df_pitch_2019 = df_pitch_2019.drop(columns = ["Team"])
df_pitch_2019 = df_pitch_2019.groupby(level=0).mean()

df_pitch_2019.head(10)

,Age,G,GS,CG,SHO,IP,H,ER,K,BB,HR,W,L,SV,BS,HLD,ERA,WHIP
Player,,,,,,,,,,,,,,,,,,
A.J. Cole,28.0,25.0,0.0,0.0,0.0,26.00,31.0,11.0,30.0,8.0,4.0,3.0,1.0,1.0,0.0,0.0,3.810,1.50
A.J. Minter,27.0,36.0,0.0,0.0,0.0,29.10,36.0,23.0,35.0,23.0,3.0,3.0,4.0,5.0,2.0,6.0,7.060,2.01
A.J. Puk,25.0,10.0,0.0,0.0,0.0,11.10,10.0,4.0,13.0,5.0,1.0,2.0,0.0,0.0,1.0,2.0,3.180,1.32
Aaron Barrett,32.0,3.0,0.0,0.0,0.0,2.10,5.0,4.0,1.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,15.430,3.86
Aaron Brooks,30.0,14.5,9.0,0.0,0.0,54.65,59.0,34.5,41.0,17.0,10.5,3.0,4.0,0.0,0.0,0.0,5.595,1.37
Aaron Bummer,27.0,58.0,0.0,0.0,0.0,67.20,43.0,16.0,60.0,24.0,4.0,0.0,0.0,1.0,2.0,27.0,2.130,0.99
Aaron Civale,25.0,10.0,10.0,0.0,0.0,57.20,44.0,15.0,46.0,16.0,4.0,3.0,4.0,0.0,0.0,0.0,2.340,1.04
Aaron Loup,32.0,4.0,0.0,0.0,0.0,3.10,2.0,0.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000,0.90
Aaron Nola,27.0,34.0,34.0,0.0,0.0,202.10,176.0,87.0,229.0,80.0,27.0,12.0,7.0,0.0,0.0,0.0,3.870,1.27


In [88]:
from sklearn.cluster import KMeans

inertia = []
k = list(range(1, 11))

for i in k:
    km = KMeans(n_clusters=i, random_state=5)
    km.fit(df_pitch_2019)
    inertia.append(km.inertia_)

# Define a DataFrame to plot the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)

:Curve   [k]   (inertia)

In [89]:
model = KMeans(n_clusters=3, random_state=7)
model.fit(df_pitch_2019)
predictions = model.predict(df_pitch_2019)
df_pitch_2019["class"] = model.labels_
df_pitch_2019.head()

,Age,G,GS,CG,SHO,IP,H,ER,K,BB,HR,W,L,SV,BS,HLD,ERA,WHIP,class
Player,,,,,,,,,,,,,,,,,,,
A.J. Cole,28.0,25.0,0.0,0.0,0.0,26.00,31.0,11.0,30.0,8.0,4.0,3.0,1.0,1.0,0.0,0.0,3.810,1.50,0
A.J. Minter,27.0,36.0,0.0,0.0,0.0,29.10,36.0,23.0,35.0,23.0,3.0,3.0,4.0,5.0,2.0,6.0,7.060,2.01,0
A.J. Puk,25.0,10.0,0.0,0.0,0.0,11.10,10.0,4.0,13.0,5.0,1.0,2.0,0.0,0.0,1.0,2.0,3.180,1.32,0
Aaron Barrett,32.0,3.0,0.0,0.0,0.0,2.10,5.0,4.0,1.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,15.430,3.86,0
Aaron Brooks,30.0,14.5,9.0,0.0,0.0,54.65,59.0,34.5,41.0,17.0,10.5,3.0,4.0,0.0,0.0,0.0,5.595,1.37,2


In [90]:
df_pitch_2018 = pd.read_csv(Path("mlb-player-stats-P2018.csv"))
df_pitch_2018.set_index(["Player"], inplace = True)
df_pitch_2018 = df_pitch_2018.drop(columns = ["Team"])
df_pitch_2018 = df_pitch_2018.groupby(level=0).mean()

df_pitch_2018.head(7)

,Age,G,GS,CG,SHO,IP,H,ER,K,BB,HR,W,L,SV,BS,HLD,ERA,WHIP
Player,,,,,,,,,,,,,,,,,,
A.J. Cole,28.0,16.0,1.0,0.0,0.0,24.05,27.5,16.5,29.5,11.0,7.5,2.0,1.0,0.0,0.5,0.0,8.66,1.790
A.J. Minter,27.0,65.0,0.0,0.0,0.0,61.10,57.0,22.0,69.0,22.0,3.0,4.0,3.0,15.0,2.0,12.0,3.23,1.290
AJ Ramos,34.0,28.0,0.0,0.0,0.0,19.20,17.0,14.0,22.0,15.0,3.0,2.0,2.0,0.0,0.0,7.0,6.41,1.630
Aaron Brooks,30.0,3.0,0.0,0.0,0.0,2.20,1.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,1.130
Aaron Bummer,27.0,37.0,0.0,0.0,0.0,31.20,40.0,15.0,35.0,10.0,1.0,0.0,1.0,0.0,1.0,2.0,4.26,1.580
Aaron Loup,32.0,29.5,0.0,0.0,0.0,19.60,24.0,10.0,22.0,7.0,2.0,0.0,0.0,0.0,0.0,5.5,4.52,1.425
Aaron Nola,27.0,33.0,33.0,0.0,0.0,212.10,149.0,56.0,224.0,58.0,17.0,17.0,6.0,0.0,0.0,0.0,2.37,0.970


In [91]:
inertia = []
k = list(range(1, 11))

# Looking for the best k
for i in k:
    km = KMeans(n_clusters=i, random_state=5)
    km.fit(df_pitch_2018)
    inertia.append(km.inertia_)

# Define a DataFrame to plot the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)

:Curve   [k]   (inertia)

In [92]:
model = KMeans(n_clusters=3, random_state=7)
model.fit(df_pitch_2018)
predictions = model.predict(df_pitch_2018)
df_pitch_2018["class"] = model.labels_
df_pitch_2018.head()

,Age,G,GS,CG,SHO,IP,H,ER,K,BB,HR,W,L,SV,BS,HLD,ERA,WHIP,class
Player,,,,,,,,,,,,,,,,,,,
A.J. Cole,28.0,16.0,1.0,0.0,0.0,24.05,27.5,16.5,29.5,11.0,7.5,2.0,1.0,0.0,0.5,0.0,8.66,1.79,0
A.J. Minter,27.0,65.0,0.0,0.0,0.0,61.10,57.0,22.0,69.0,22.0,3.0,4.0,3.0,15.0,2.0,12.0,3.23,1.29,2
AJ Ramos,34.0,28.0,0.0,0.0,0.0,19.20,17.0,14.0,22.0,15.0,3.0,2.0,2.0,0.0,0.0,7.0,6.41,1.63,0
Aaron Brooks,30.0,3.0,0.0,0.0,0.0,2.20,1.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,1.13,0
Aaron Bummer,27.0,37.0,0.0,0.0,0.0,31.20,40.0,15.0,35.0,10.0,1.0,0.0,1.0,0.0,1.0,2.0,4.26,1.58,0


In [93]:
df_pitch_combined = pd.concat([df_pitch_2018, df_pitch_2019])

In [94]:
df_pitch_combined = df_pitch_combined.groupby(level=0).mean()
df_pitch_combined = df_pitch_combined.drop(columns = ["class"])

In [95]:
inertia = []
k = list(range(1, 11))

# Looking for the best k
for i in k:
    km = KMeans(n_clusters=i, random_state=5)
    km.fit(df_pitch_combined)
    inertia.append(km.inertia_)

# Define a DataFrame to plot the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)

:Curve   [k]   (inertia)

In [96]:
model = KMeans(n_clusters=3, random_state=7)
model.fit(df_pitch_combined)
predictions = model.predict(df_pitch_combined)
df_pitch_combined["class"] = model.labels_
df_pitch_combined.head()

,Age,G,GS,CG,SHO,IP,H,ER,K,BB,HR,W,L,SV,BS,HLD,ERA,WHIP,class
Player,,,,,,,,,,,,,,,,,,,
A.J. Cole,28.0,20.5,0.5,0.0,0.0,25.025,29.25,13.75,29.75,9.5,5.75,2.5,1.0,0.5,0.25,0.0,6.235,1.645,1
A.J. Minter,27.0,50.5,0.0,0.0,0.0,45.100,46.50,22.50,52.00,22.5,3.00,3.5,3.5,10.0,2.00,9.0,5.145,1.650,2
A.J. Puk,25.0,10.0,0.0,0.0,0.0,11.100,10.00,4.00,13.00,5.0,1.00,2.0,0.0,0.0,1.00,2.0,3.180,1.320,1
AJ Ramos,34.0,28.0,0.0,0.0,0.0,19.200,17.00,14.00,22.00,15.0,3.00,2.0,2.0,0.0,0.00,7.0,6.410,1.630,1
Aaron Barrett,32.0,3.0,0.0,0.0,0.0,2.100,5.00,4.00,1.00,4.0,1.00,0.0,0.0,0.0,0.00,0.0,15.430,3.860,1


# importing batter data

In [97]:
df_bat_2019 = pd.read_csv(Path("mlb-player-stats-Batters.csv"))
df_bat_2019 = df_bat_2019.drop(columns = ["Player"])

df_bat_2019 = df_bat_2019.groupby(by = "Team").mean()
df_bat_2019 = df_bat_2019[["SO", "AVG", "HBP", "SH", "HR", "RBI", "AVG", "R", "H"]]
df_bat_2019.head()

,SO,AVG,HBP,SH,HR,RBI,AVG,R,H
Team,,,,,,,,,
ARI,57.857143,0.232333,3.333333,0.238095,10.238095,35.952381,0.232333,37.714286,66.000000
ATL,65.400000,0.237550,2.900000,0.150000,12.450000,40.500000,0.237550,41.950000,70.050000
BAL,59.583333,0.210417,2.958333,0.791667,8.875000,29.083333,0.210417,30.333333,57.416667
BOS,65.285714,0.229381,2.333333,0.857143,11.666667,40.761905,0.229381,42.809524,73.952381
CHC,57.260870,0.221609,3.521739,0.304348,11.086957,33.043478,0.221609,34.869565,58.217391


In [98]:
df_bat_2018 = pd.read_csv(Path("mlb-player-stats-Batters2018.csv"))
df_bat_2018 = df_bat_2018.drop(columns = ["Player"])

df_bat_2018 = df_bat_2018.groupby(by = "Team").mean()
df_bat_2018 = df_bat_2018[["SO", "AVG", "HBP", "SH", "HR", "RBI", "AVG", "R", "H"]]
df_bat_2018.head()

,SO,AVG,HBP,SH,HR,RBI,AVG,R,H
Team,,,,,,,,,
ARI,60.272727,0.216318,2.363636,0.454545,8.000000,29.227273,0.216318,30.909091,56.681818
ATL,47.291667,0.208625,2.750000,0.541667,7.250000,29.375000,0.208625,31.083333,58.750000
BAL,50.142857,0.220679,2.035714,0.428571,6.714286,21.178571,0.220679,22.107143,46.964286
BOS,62.150000,0.237950,2.750000,0.350000,10.400000,41.300000,0.237950,43.600000,75.250000
CHC,62.500000,0.251000,3.900000,0.650000,8.250000,35.350000,0.251000,37.400000,71.200000


In [99]:
df_bat_combined = pd.concat([df_bat_2018, df_bat_2019])
df_bat_combined = df_bat_combined.groupby(level=0).mean()
df_bat_combined.head()

,SO,AVG,HBP,SH,HR,RBI,AVG,R,H
Team,,,,,,,,,
ARI,59.064935,0.224326,2.848485,0.346320,9.119048,32.589827,0.224326,34.311688,61.340909
ATL,56.345833,0.223088,2.825000,0.345833,9.850000,34.937500,0.223088,36.516667,64.400000
BAL,54.863095,0.215548,2.497024,0.610119,7.794643,25.130952,0.215548,26.220238,52.190476
BOS,63.717857,0.233665,2.541667,0.603571,11.033333,41.030952,0.233665,43.204762,74.601190
CHC,59.880435,0.236304,3.710870,0.477174,9.668478,34.196739,0.236304,36.134783,64.708696


In [100]:
df_hou = df_bat_combined.loc["HOU"]
df_hou = pd.DataFrame(df_hou)
df_hou = df_hou.transpose()
df_hou.rename(columns = {
    "SO" : "Astros-SO", 
    "AVG" : "Astros-AVG", 
    "HBP" : "Astros-HBP", 
    "SH" : "Astros-SH", 
    "HR" : "Astros-HR", 
    "RBI" : "Astros-RBI", 
    "AVG" : "Astros-AVG", 
    "R" : "Astros-R", 
    "H" : "Astros-H"
}, inplace=True)
df_hou.head()

,Astros-SO,Astros-AVG,Astros-HBP,Astros-SH,Astros-HR,Astros-RBI,Astros-AVG,Astros-R,Astros-H
HOU,56.95,0.239085,3.096429,0.564286,11.982143,40.289286,0.239085,41.805952,71.295238


In [101]:
df_hou['key'] = 1
merge['key'] = 1
df = pd.merge(df_hou, merge, on='key')
del df['key']

# combining all the dataframes

In [103]:
merge = pd.merge(df_combined, df_bat_combined, left_index=True, right_index=True)
merge

,Result,Starting Pitcher,Opposing Pitcher,SO,AVG,HBP,SH,HR,RBI,AVG,R,H
ARI,1,Gerrit Cole,Kris Medlen,59.064935,0.224326,2.848485,0.346320,9.119048,32.589827,0.224326,34.311688,61.340909
ARI,0,Charlie Morton,Zack Greinke,59.064935,0.224326,2.848485,0.346320,9.119048,32.589827,0.224326,34.311688,61.340909
ARI,0,Justin Verlander,Matt Koch,59.064935,0.224326,2.848485,0.346320,9.119048,32.589827,0.224326,34.311688,61.340909
ARI,0,Dallas Keuchel,Robbie Ray,59.064935,0.224326,2.848485,0.346320,9.119048,32.589827,0.224326,34.311688,61.340909
ARI,1,Charlie Morton,Zack Godley,59.064935,0.224326,2.848485,0.346320,9.119048,32.589827,0.224326,34.311688,61.340909
...,...,...,...,...,...,...,...,...,...,...,...,...
TOR,1,Charlie Morton,Ryan Borucki,55.851190,0.216542,1.973214,0.380952,9.020833,26.663690,0.216542,27.708333,50.788690
TOR,1,Dallas Keuchel,Marco Estrada,55.851190,0.216542,1.973214,0.380952,9.020833,26.663690,0.216542,27.708333,50.788690
TOR,1,Dallas Keuchel,Marco Estrada,55.851190,0.216542,1.973214,0.380952,9.020833,26.663690,0.216542,27.708333,50.788690
TOR,1,Joshua James,Sam Gaviglio,55.851190,0.216542,1.973214,0.380952,9.020833,26.663690,0.216542,27.708333,50.788690


In [109]:
df_hou['key'] = 1
merge['key'] = 1
df = pd.merge(df_hou, merge, on='key')
del df['key']
merge = df

In [20]:
random_number = list(range(len(merge)))

In [104]:
opp_pitch_class = []
opp_pitch_class_2 = []
opp_pitch_class_3 = []

for number in random_number:
    if merge["Opposing Pitcher"][number] in df_pitch_combined.index:
        name = merge["Opposing Pitcher"][number]
        opp_pitch_class.append(df_pitch_combined.loc[name]["ERA"])
        opp_pitch_class_2.append(df_pitch_combined.loc[name]["WHIP"])
        opp_pitch_class_3.append(df_pitch_combined.loc[name]["K"])
    else:
        print(merge["Opposing Pitcher"][number])

In [29]:
opp_pitch_class = []
opp_pitch_class_2 = []
opp_pitch_class_3 = []

for number in random_number:
    if merge["Opposing Pitcher"][number] in df_pitch_combined.index:
        name = merge["Opposing Pitcher"][number]
        opp_pitch_class.append(df_pitch_combined.loc[name]["class"])
    else:
        print(merge["Opposing Pitcher"][number])

In [105]:
start_pitch_class = []
start_pitch_class_2 = []
start_pitch_class_3 = []

for number in random_number:
    if merge["Starting Pitcher"][number] in df_pitch_combined.index:
        name = merge["Starting Pitcher"][number]
        start_pitch_class.append(df_pitch_combined.loc[name]["ERA"])
        start_pitch_class_2.append(df_pitch_combined.loc[name]["WHIP"])
        start_pitch_class_3.append(df_pitch_combined.loc[name]["K"])
    else:
        print(merge["Starting Pitcher"][number])

In [30]:
start_pitch_class = []
start_pitch_class_2 = []
start_pitch_class_3 = []

for number in random_number:
    if merge["Starting Pitcher"][number] in df_pitch_combined.index:
        name = merge["Starting Pitcher"][number]
        start_pitch_class.append(df_pitch_combined.loc[name]["class"])
    else:
        print(merge["Starting Pitcher"][number])

In [106]:
merge["ERA-Starting "] = start_pitch_class
merge["ERA-Opposing"] = opp_pitch_class

merge["WHIP-Starting"] = start_pitch_class_2
merge["WHIP-Opposing"] = opp_pitch_class_2

merge["K-Starting"] = start_pitch_class_2
merge["K-Opposing"] = opp_pitch_class_2

In [31]:
merge["Starting Pitcher"] = start_pitch_class
merge["Opposing Pitcher"] = opp_pitch_class

In [107]:
merge.drop(["Starting Pitcher"], axis=1, inplace=True)
merge.drop(["Opposing Pitcher"], axis=1, inplace=True)

In [110]:
merge.head()

,Astros-SO,Astros-AVG,Astros-HBP,Astros-SH,Astros-HR,Astros-RBI,Astros-AVG,Astros-R,Astros-H,Result,...,RBI,AVG,R,H,ERA-Starting,ERA-Opposing,WHIP-Starting,WHIP-Opposing,K-Starting,K-Opposing
0,56.95,0.239085,3.096429,0.564286,11.982143,40.289286,0.239085,41.805952,71.295238,1,...,32.589827,0.224326,34.311688,61.340909,2.690,15.7500,0.96,3.2500,0.96,3.2500
1,56.95,0.239085,3.096429,0.564286,11.982143,40.289286,0.239085,41.805952,71.295238,0,...,32.589827,0.224326,34.311688,61.340909,3.090,3.0850,1.12,1.0450,1.12,1.0450
2,56.95,0.239085,3.096429,0.564286,11.982143,40.289286,0.239085,41.805952,71.295238,0,...,32.589827,0.224326,34.311688,61.340909,2.550,6.6500,0.85,1.4350,0.85,1.4350
3,56.95,0.239085,3.096429,0.564286,11.982143,40.289286,0.239085,41.805952,71.295238,0,...,32.589827,0.224326,34.311688,61.340909,3.745,4.1350,1.34,1.3450,1.34,1.3450
4,56.95,0.239085,3.096429,0.564286,11.982143,40.289286,0.239085,41.805952,71.295238,1,...,32.589827,0.224326,34.311688,61.340909,3.090,4.9525,1.12,1.4525,1.12,1.4525


In [121]:
for col in merge.columns: 
    print(col) 

Astros-SO
Astros-AVG
Astros-HBP
Astros-SH
Astros-HR
Astros-RBI
Astros-AVG
Astros-R
Astros-H
Result
SO
AVG
HBP
SH
HR
RBI
AVG
R
H
ERA-Starting 
ERA-Opposing
WHIP-Starting
WHIP-Opposing
K-Starting
K-Opposing


# Running some tests

In [111]:
X = merge.copy()
X.drop(["Result"], axis=1, inplace=True)

y = merge["Result"].ravel()
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=87)

X_scaler = StandardScaler()
X_scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Model 1 = Random Forest Classifier

In [112]:
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

model = BalancedRandomForestClassifier(random_state=1)
model = model.fit(X_train, y_train)

y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

predictions = model.predict(X_test)
results = pd.DataFrame({"Prediction": predictions,"Actual": y_test}).reset_index(drop=True)
confusion_matrix(y_test, predictions)

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.44      0.57      0.55      0.50      0.56      0.31        21
          1       0.67      0.55      0.57      0.60      0.56      0.31        33

avg / total       0.58      0.56      0.56      0.56      0.56      0.31        54



In [113]:
predictions = model.predict(X_test)
results = pd.DataFrame({"Prediction": predictions,
                        "Actual": y_test}).reset_index(drop=True)
balanced_accuracy_score(y_test, predictions)

0.5584415584415584

## Model 2 = Decision Tree 

In [114]:
from sklearn import tree, preprocessing
from sklearn.tree import DecisionTreeClassifier

model = tree.DecisionTreeClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.45      0.48      0.47        21
           1       0.66      0.64      0.65        33

    accuracy                           0.57        54
   macro avg       0.56      0.56      0.56        54
weighted avg       0.58      0.57      0.58        54



In [115]:
predictions = model.predict(X_test)
results = pd.DataFrame({"Prediction": predictions,
                        "Actual": y_test}).reset_index(drop=True)
balanced_accuracy_score(y_test, predictions)

0.5562770562770563

## Model 3, neurons model(still needs work)

In [ ]:
# X_scaler = StandardScaler()
# X_scaler.fit(X_train)
# X_train_scaled = X_scaler.transform(X_train)
# X_test_scaled = X_scaler.transform(X_test)

In [130]:
y = merge["Result"].values
y = y.reshape(-1, 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)
X_scaler = StandardScaler()
X_scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [136]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model = Sequential()
nn = Sequential()

num_units= 24
dropout_pct = 0.2

# Layer #1 - LSTM
model.add(LSTM(units=num_units, 
               return_sequences=True, 
               input_shape=(X_train.shape[1], 1))
         )
# Layer #1 dropout
model.add(Dropout(dropout_pct))

# Layer #2- LSTM
model.add(LSTM(units=num_units, return_sequences=True))
# Layer #2 dropout
model.add(Dropout(dropout_pct))

# Layer #3- LSTM
model.add(LSTM(units=num_units))
# Layer #3 dropout
model.add(Dropout(dropout_pct))

# Output
model.add(Dense(1))

In [138]:
model.compile(optimizer="adam", loss="mean_squared_error")
# model.summary()


In [141]:
model = model.fit(X_train_scaled, y_train, epochs=100, batch_size=(None, 64, 1))

TypeError: unsupported operand type(s) for /: 'int' and 'tuple'

In [157]:
model_loss, model_accuracy = nn.evaluate(
    X_test_scaled, y_test, verbose=2
)

2/2 - 0s - loss: 0.9939 - accuracy: 0.5000
Loss: 0.9938737154006958, Accuracy: 0.5


## Model 4 - K Neighbors

In [116]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()

In [117]:
knn.fit(X_train,y_train)
predictions = knn.predict(X_test)
results = pd.DataFrame({"Prediction": predictions,
                        "Actual": y_test}).reset_index(drop=True)
balanced_accuracy_score(y_test, predictions)

0.5584415584415584

## Model 5 - XGBoost(was working, now not working, was best one)

In [122]:
from xgboost import XGBClassifier

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions)
accuracy

ValueError: feature_names must be unique

In [123]:
X_train

,Astros-SO,Astros-AVG,Astros-HBP,Astros-SH,Astros-HR,Astros-RBI,Astros-AVG,Astros-R,Astros-H,SO,...,RBI,AVG,R,H,ERA-Starting,ERA-Opposing,WHIP-Starting,WHIP-Opposing,K-Starting,K-Opposing
111,56.95,0.239085,3.096429,0.564286,11.982143,40.289286,0.239085,41.805952,71.295238,63.288043,...,36.794565,0.226137,38.647826,65.056522,2.690,4.44000,0.960,1.280,0.960,1.280
140,56.95,0.239085,3.096429,0.564286,11.982143,40.289286,0.239085,41.805952,71.295238,55.341385,...,27.496779,0.225651,28.735910,54.561192,2.550,2.52000,0.850,1.210,0.850,1.210
122,56.95,0.239085,3.096429,0.564286,11.982143,40.289286,0.239085,41.805952,71.295238,63.288043,...,36.794565,0.226137,38.647826,65.056522,2.690,4.18500,0.960,1.295,0.960,1.295
153,56.95,0.239085,3.096429,0.564286,11.982143,40.289286,0.239085,41.805952,71.295238,55.341385,...,27.496779,0.225651,28.735910,54.561192,2.690,3.79000,0.960,1.190,0.960,1.190
21,56.95,0.239085,3.096429,0.564286,11.982143,40.289286,0.239085,41.805952,71.295238,63.717857,...,41.030952,0.233665,43.204762,74.601190,3.275,3.25500,1.275,0.975,1.275,0.975
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,56.95,0.239085,3.096429,0.564286,11.982143,40.289286,0.239085,41.805952,71.295238,55.341385,...,27.496779,0.225651,28.735910,54.561192,2.550,4.34875,0.850,2.930,0.850,2.930
6,56.95,0.239085,3.096429,0.564286,11.982143,40.289286,0.239085,41.805952,71.295238,54.863095,...,25.130952,0.215548,26.220238,52.190476,2.690,5.61000,0.960,1.370,0.960,1.370
143,56.95,0.239085,3.096429,0.564286,11.982143,40.289286,0.239085,41.805952,71.295238,55.341385,...,27.496779,0.225651,28.735910,54.561192,2.550,5.46000,0.850,1.520,0.850,1.520
187,56.95,0.239085,3.096429,0.564286,11.982143,40.289286,0.239085,41.805952,71.295238,78.007895,...,37.531579,0.229646,39.740789,68.806579,3.860,3.39500,1.170,1.030,1.170,1.030


# Defining Functions